In [1]:
import pandas as pd
import trollcamp

config = trollcamp.load_config()

batch_name = config['batch_name']
lol_season = config['lol_season']
token_key = config['token_key']
my_chunk_idx = config['my_chunk_idx']
db_config = config['db_config']

In [2]:
def get_matchlist_with_last_time(account_id, db_last_game_creation):
    history = {'accountId': account_id, 'matches': []}
    last_game_creation = 0
    
    begin_idx = 0
    end_idx = 100
    
    run = True
    while run:
        (result, res_data) = trollcamp.call_api_matchlist(account_id, begin_idx, end_idx, token_key)
        if result == True:
            matches = res_data['matches']
            if len(matches) < 1:
                run = False
                break
            for match in matches:
                game_season = match['season']
                if game_season < lol_season:
                    run = False
                    break
                game_creation = match['timestamp']
                if game_creation <= db_last_game_creation:
                    run = False
                    break
                if begin_idx == 0:
                    last_game_creation = trollcamp.get_bigger(game_creation, last_game_creation)
                history['matches'].append(match)
        else:
            break
                
        begin_idx += 100
        end_idx +=100
        
    return (history, last_game_creation)

def get_matchlist(account_id):
    history = {'accountId': account_id, 'matches': []}
    last_game_creation = 0
    
    (result, res_data) = trollcamp.call_api_matchlist(account_id, 0, 100, token_key)
    if result == True:
        matches = res_data['matches']
        if len(matches) > 0:
            for match in matches:
                game_season = match['season']
                if game_season < lol_season:
                    run = False
                    break
                last_game_creation = trollcamp.get_bigger(match['timestamp'], last_game_creation)
                history['matches'].append(match)
        
    return (history, last_game_creation)

In [3]:
players_df = trollcamp.read_csv("%s/players/players-%d.csv" % (batch_name, my_chunk_idx), config)   
players_df = players_df.set_index('account_id')
#players_df

In [4]:
cnx = trollcamp.db_connect(db_config)

In [ ]:
# 테스트로 3개만 진행한다. 실제로는 temp_count 변수 관련 모두 삭제하고 실행할 것!

file_name = "%s.json" % batch_name

#temp_count = 0

for account_id, values in players_df.iterrows():
    (db_exist, db_last_game_creation) = trollcamp.db_check_player_info(cnx, account_id)
    history = dict()
    last_game_creation = 0
    
    if db_exist == True:
        (history, last_game_creation) = get_matchlist_with_last_time(account_id, db_last_game_creation)
    else:
        (history, last_game_creation) = get_matchlist(account_id)
            
    if last_game_creation > 0:
        trollcamp.db_set_player_info(cnx, db_exist, account_id, last_game_creation)
        dir_path = "players/%s" % account_id
        trollcamp.write_json_with_check_directory(history, dir_path, file_name, config)
        print("new file : %s/%s" % (dir_path, file_name))
       
    #temp_count += 1
    #if temp_count >= 3:
        #break
    
print("Completed.") 

In [6]:
trollcamp.db_close(cnx)